In [11]:

#A skeleton for implementing Naive Bayes Classifier in Python.
## Author: Md Faisal Kabir
## Spring 2025

import numpy
import random
import time
import pandas

# Change these file names to swap programs
trainingFile = "irisTraining.txt"
testingFile = "irisTesting.txt"

# Specify if the attributes are continuous (cont) or categorical (cat)
attributeType = "cont" 

# Xtrain = numpy.loadtxt(trainingFile)
XtrainPandas = pandas.read_csv(trainingFile, sep=' ')

n = XtrainPandas.shape[0]
d = XtrainPandas.shape[1]-1
print(n, d)

100 4


In [36]:
#Training... Collect mean and standard deviation for each dimension for each class..
#Also, calculate P(C+) and P(C-)

# value trackers for classification -1
col_mean_vals_cn1 = []  
col_stddev_vals_cn1 = []

# value trackers for classification +1
col_mean_vals_cp1 = []  
col_stddev_vals_cp1 = []

if(attributeType == "cont"):
    #print(XtrainPandas)
    SplitXtrain = dict(tuple(XtrainPandas.groupby('Classification')))
    # print(SplitXtrain[-1])
    # print(SplitXtrain[1])

    for col in SplitXtrain[-1]:
        if(col == 'Classification'):
            print(f"skipping {col}")
            break
        col_mean_vals_cn1.append(SplitXtrain[-1][col].mean())
        col_stddev_vals_cn1.append(SplitXtrain[-1][col].std())
        print(f"{col} for classification -1: mean {col_mean_vals_cn1[len(col_mean_vals_cn1)-1]}" \
            + f", stddev {col_stddev_vals_cn1[len(col_stddev_vals_cn1)-1]}\n")
    
    print('-----------------------------')
    
    for col in SplitXtrain[1]:
        if(col == 'Classification'):
            print(f"skipping {col}")
            break
        col_mean_vals_cp1.append(SplitXtrain[1][col].mean())
        col_stddev_vals_cp1.append(SplitXtrain[1][col].std())
        print(f"{col} for classification 1: mean {col_mean_vals_cp1[len(col_mean_vals_cp1)-1]}" \
            + f", stddev {col_stddev_vals_cp1[len(col_stddev_vals_cp1)-1]}\n")


Sepal_Length for classification -1: mean 5.771212121212121, stddev 0.9419725868157492

Sepal_Width for classification -1: mean -1.3181818181818181, stddev 0.7743799685324617

Petal_Length for classification -1: mean 3.548484848484849, stddev 2.0803285557254894

Petal_Width for classification -1: mean 0.22272727272727275, stddev 1.0916177768016468

skipping Classification
-----------------------------
Sepal_Length for classification 1: mean 5.847058823529412, stddev 0.5124127681076909

Sepal_Width for classification 1: mean -1.388235294117647, stddev 0.31981500891718434

Petal_Length for classification 1: mean 3.005882352941176, stddev 2.5404534523024638

Petal_Width for classification 1: mean 1.3, stddev 0.21602468994692872

skipping Classification


In [ ]:
#Testing .....
Xtest = numpy.loadtxt(testingFile)
nn = Xtest.shape[0] # Number of points in the testing data.

tp = 0 #True Positive
fp = 0 #False Positive
tn = 0 #True Negative
fn = 0 #False Negative



In [ ]:
#Iterate over all points in testing data
  #For each point find the P(C+|Xi) and P(C-|Xi) and decide if the point belongs to C+ or C-..
  #Recall we need to calculate P(Xi|C+)*P(C+) ..
  #P(Xi|C+) = P(Xi1|C+) * P(Xi2|C+)....P(Xid|C+)....Do the same for P(Xi|C-)
  #Now that you've calculate P(Xi|C+) and P(Xi|C-), we can decide which is higher 
  #P(Xi|C-)*P(C-) or P(Xi|C-)*P(C-) ..
  #increment TP,FP,FN,TN accordingly, remember the true lable for the ith point is in Xtest[i,d]

#}

#Calculate all the measures required..